In [35]:
from icecube import icetray, dataio, dataclasses, simclasses, clsim
from icecube.icetray import I3Units, OMKey, I3Frame
from icecube.dataclasses import ModuleKey
from os.path import expandvars
import scipy.constants as spc
import scipy as sc
import numpy as np
import matplotlib.pylab as plt
from scipy import stats
from scipy.optimize import minimize
from scipy.stats.distributions import chi2
import scipy

infile = '/data/p-one/akatil/step_4_medium_water/NuTau_NuE_20Events/step_4_'+str(713)+'_medium_water_custom_mDOM_noise.i3.gz'

In [36]:
'''

Functions used

'''

def gaussian(x, pos, wid, amp):
    y = amp*np.exp(-4*np.log(2)*((x-pos)/(wid))**2)
    return y

def biGauss(x, pos, wid, r, amp):
    mask = x < pos

    y_all = ([])
    for i in range(0, len(mask)):

        if mask[i] == True:
            m = 1
            nm = 0
        else:
            m = 0
            nm = 1
        if r != 0:
            y1 = gaussian(x[i],pos,r*wid/(r+1),amp)*m
            y2 = gaussian(x[i],pos,wid/(r+1),amp)*nm
            y = y1 + y2
        else:
            y = gaussian(x[i],pos,wid, amp)*nm

        y_all = np.append(y_all, y)
    return y_all

def double_peak(x, pos1, wid1, r1, amp1, pos2, wid2, r2, amp2):
    b1 = biGauss(x, pos1, wid1, r1, amp1)
    b2 = biGauss(x, pos2, wid2, r2, amp2)
    b = np.append(b1, b2)
    return b1+b2

def log_likelihood_biGauss(theta, n, x):
    pos, wid, r, amp = theta
    model = biGauss(x, pos, wid, r, amp)
    L = model - (n*np.log(model))
    #print('*****************One Peak***************')
    #print(theta, np.sum(L))
    return np.sum(L)

def log_likelihood_doublePeak(theta, n, x):
    pos1, wid1, r1, amp1, pos2, wid2, r2, amp2 = theta
    model = double_peak(x, pos1, wid1, r1, amp1, pos2, wid2, r2, amp2)
    L = model - (n*np.log(model))
    #print('*****************Double Peak***************')
    #print(theta, np.sum(L))
    return np.sum(L)

def likelihood_ratio_doublePeak(x, n, pos1, wid1, r1, amp1, pos2, wid2, r2, amp2):
    model = double_peak(x, pos1, wid1, r1, amp1, pos2, wid2, r2, amp2)
    val = model - n + (n*np.log(n/model))
    #print('log - ', n/model, 'n - ', n)
    return np.sum(val)

def likelihood_ratio_biGauss(x, n, pos, wid, r, amp):
    model = biGauss(x, pos, wid, r, amp)
    val = model - n + (n*np.log(n/model))
    #print('log - ', n/model, 'n - ', n)
    return np.sum(val)

In [37]:
'''

Loading the geometry

'''

gcd_file = dataio.I3File('/home/users/akatil/P-ONE/GCD_files/PONE_Phase1.i3.gz')
cframe = gcd_file.pop_frame()
geometry = cframe["I3Geometry"]
omgeo = geometry.omgeo
print('loaded geometry')

loaded geometry


In [38]:
def likelihoodfit(frame):
    print('Likelihood fit function called')
    
    tau_timeDiff = ([])
    tau_pVal = ([])
    tau_LRR = ([])
    
    e_timeDiff = ([])
    e_pVal = ([])
    e_LRR = ([])


    #file = dataio.I3File(str(infile))
    
    mctree = frame["I3MCTree"]
    primary = mctree.primaries
    lepton = dataclasses.I3MCTree.first_child(mctree, primary[0].id)

    mcpeMap = frame['MCPESeriesMap']
    NoiseMap = frame['NoiseSeriesMap'] 

    for omkey in mcpeMap.keys():
        oKey = omgeo.get(omkey)

        '''
        Obtaining the timeList
        '''
        noise_mcpeList = noiseMap[omkey]
        noise_timeList = np.array([mcpe.time for mcpe in noise_mcpeList])
        mcpeList = mcpeMap[omkey]
        timeList = np.array([mcpe.time for mcpe in mcpeList])
        tot_timeList = np.append(timeList, noise_timeList)


        '''
        Removing DOMs with hits less than 250 Hits
        '''
        if len(tot_timeList) < 250:
            continue


        '''
        Calculating the mean and removing the tails
        '''

        mean = tot_timeList.mean()

        select_time = tot_timeList[(tot_timeList > mean-50) & (tot_timeList < mean+50)]
        new_mean = select_time.mean()

        bins = np.arange(min(select_time), max(select_time), 1)
        max_hitTimes = select_time[(select_time > (new_mean-40))&(select_time < (new_mean+40))]
    

        #[using zscore to remove the effect of outliers from the analysis]
            
        z = stats.zscore(max_hitTimes)
            
        max_hitTimes = max_hitTimes[(z>-1.6)&(z < 1.2)]
        new_mean = max_hitTimes.mean()
            
        #Shifting mean to zero
            
        timestamps = max_hitTimes - new_mean
        final_mean = timestamps.mean()
            
        if len(max_hitTimes) < 250:
            continue
            
        '''
        Histogramming the data from simulation
        '''

        bins = np.arange(min(timestamps), max(timestamps), 1)
        num, bin_edges = np.histogram(timestamps, bins=bins)
        bin_centers = (bin_edges[:-1]+bin_edges[1:])/2
            
        '''
        Fitting bifurcated Gaussian and double bifurcated gaussian to 
        the mcpe hit time distributions for both tau and electron.
        '''
            
        #Single Peak
            
        nll = lambda *args: log_likelihood_biGauss(*args)
        initial_biGauss = np.array([final_mean, 50, 5, max(num)])
        bnds_biGauss = ((min(bin_centers), max(bin_centers)), (0, 100), (0, 10), (0, 1e6))
        soln_biGauss = minimize(log_likelihood_biGauss, initial_biGauss, 
                                args=(num, bin_centers),
                                method='Powell', 
                                bounds = bnds_biGauss)
            
        #Double Peak
            
        nll = lambda *args: log_likelihood_doublePeak(*args)
        initial_doublePeak = np.array([min(bin_centers)+10, 20, 1, max(num), final_mean, 20, 1, max(num)])
        bnds_doublePeak = ((min(bin_centers), final_mean), (0, 100), (0, 10), (0, 1e6),
                            (final_mean, max(bin_centers)), (0, 100), (0, 10), (0,1e6))
        soln_doublePeak = minimize(log_likelihood_doublePeak, initial_doublePeak, 
                                    args=(num, bin_centers),
                                    method='Powell',
                                    bounds=bnds_doublePeak)
            
        '''
        Removing DOMs whose minimization is not successful
        '''
        if soln_biGauss.success == False or soln_doublePeak.success == False:
            print('Removing DOMs whose minimization is not successful')
            continue
                
        '''
        Calculating the Likelihood ratio for bifurcated gaussian 
        and double double bifurcated gaussian
        '''
        LR_biGauss = likelihood_ratio_biGauss(bin_centers[num>0], num[num>0], soln_biGauss.x[0],
                                              soln_biGauss.x[1], soln_biGauss.x[2], soln_biGauss.x[3])
        LR_doublePeak = likelihood_ratio_doublePeak(bin_centers[num>0], num[num>0], soln_doublePeak.x[0], 
                                                    soln_doublePeak.x[1],soln_doublePeak.x[2], 
                                                    soln_doublePeak.x[3], soln_doublePeak.x[4],
                                                    soln_doublePeak.x[5], soln_doublePeak.x[6], 
                                                    soln_doublePeak.x[7])
            
        '''
        Calculating the p-value using the likelihood ratio
        '''
        pVal_biGauss = chi2.sf(LR_biGauss, len(num) - 4)
        pVal_doublePeak = chi2.sf(LR_doublePeak, len(num) - 8)
        
        '''
        Are there any messed up p-values?
        '''
        if pVal_biGauss != pVal_biGauss:
            print('BiGauss is not well defined - ', str(lepton.type))
            print('Minimisation - ', soln_biGauss.success)
            print('Degrees of Freedom - ', len(num) - 4)
            print('Log Likelihood - ', LR_biGauss)
        if pVal_doublePeak != pVal_doublePeak:
            print('double peak is not well defined - ', str(lepton.type))
            print('Minimisation - ', soln_doublePeak.success)
            print('Degrees of Freedom - ', len(num) - 8)
            print('Log Likelihood - ', LR_doublePeak)
                
        '''
        (x, y) values for the fit
        '''
        x = np.linspace(min(bin_centers), max(bin_centers), 1000)
        #x = np.linspace(0, max(bin_centers)+1e5, 1000)
        y_biGauss = biGauss(x, soln_biGauss.x[0],
                                soln_biGauss.x[1], soln_biGauss.x[2], soln_biGauss.x[3])
        y_doublePeak = double_peak(x, soln_doublePeak.x[0], soln_doublePeak.x[1],
                                            soln_doublePeak.x[2], soln_doublePeak.x[3], soln_doublePeak.x[4],
                                            soln_doublePeak.x[5], soln_doublePeak.x[6], soln_doublePeak.x[7])
                
        '''
        Calculating the time difference and p-value ratio of bigauss and double peak
        '''
        timeDifference_doublePeak = soln_doublePeak.x[4] - soln_doublePeak.x[0]
        pVal_ratio = pVal_doublePeak/pVal_biGauss
        LRR = LR_doublePeak/LR_biGauss

        '''
        Note - Minimize for double peak can sometimes return a large 
        time difference despite the fact the data actually contains only one peak
        Accounting for that below using Likelihood ratios. If the Likehihood ratios of
        the single peak and double peak are similar the timedifference will be forced to 
        be zero
            
        '''
            
        if LRR >= 0.85 and LRR <=1.3:
            print('Time Difference before -', timeDifference_doublePeak)
            if abs(timeDifference_doublePeak) < 100:
                plt.figure(figsize=(10,9))
                _ = plt.hist(timestamps, bins=bins, histtype='step')
                plt.title(str(lepton.type)+' timeDifference < 100 ' + str(abs(LR_doublePeak/LR_biGauss)))
                plt.plot(x, y_biGauss, '--', c = 'r')
                plt.plot(x, y_doublePeak, '--', c = 'k')
                plt.axvline(final_mean, c = 'b')
                timeDifference_doublePeak = 0.
                
        '''
        Checking if there are any terrible fits
        '''
        amp1 = soln_doublePeak.x[3]
        amp2 = soln_doublePeak.x[7]
            
        if amp1/amp2 < 1/4 and amp1/amp2 > 4:
            print('Removing terrible fits')
                
        if amp1 < 0 or amp2 < 0:
            print('Error in amp')
            
        '''
        Separating the time difference calculated above and appending the values
        '''

        '''
        Tau
        '''
        if lepton.type == 15 or lepton.type == -15:
            tau_timeDiff = np.append(tau_timeDiff, timeDifference_doublePeak)
            tau_pVal = np.append(tau_pVal, pVal_ratio)
            tau_LRR = np.append(tau_LRR, LRR)
            #plt.title('E')

        '''
        Electron and Neutral Current
        '''

        if lepton.type == 11 or lepton.type == -11 or lepton.type == 12 or lepton.type == -12 or lepton.type == 16 or lepton.type == -16:

            e_timeDiff = np.append(e_timeDiff, timeDifference_doublePeak)
            e_pVal = np.append(e_pVal, pVal_ratio)
            e_LRR = np.append(e_LRR, LRR)
            
                
        

In [39]:
file = dataio.I3File(str(infile))

for frame in file:
    print('Starting')
    numHitsinDOM = ([])
    numHits = ([])
    mctree = frame["I3MCTree"]
    primary = mctree.primaries
    lepton = dataclasses.I3MCTree.first_child(mctree, primary[0].id)

    mcpeMap = frame['MCPESeriesMap']
    noiseMap = frame['NoiseSeriesMap']

    #looping through doms that have physics hits
    for omkey in mcpeMap.keys():
        oKey = omgeo.get(omkey)

        '''
        Obtaining the timeList
        '''
        noise_mcpeList = noiseMap[omkey]
        noise_timeList = np.array([mcpe.time for mcpe in noise_mcpeList])
        mcpeList = mcpeMap[omkey]
        timeList = np.array([mcpe.time for mcpe in mcpeList])
        tot_timeList = np.append(timeList, noise_timeList)
        numHitsinDOM = np.append(tot_timeList, noise_timeList)

        #print(numHitsinDOM)

    numHits = np.append(numHits, sum(numHitsinDOM))
    log_numHits = np.log10(numHits[numHits > 0])
    print(log_numHits)
    if log_numHits > 5.5 and log_numHits <= 6.0:
        likelihoodfit(frame)

Starting
[6.92615955]
Starting
[8.01509652]
Starting
[7.74887335]
Starting
[5.31716524]
Starting
[8.31380244]
Starting
[4.64522921]
Starting
[8.085902]
Starting
[8.24826947]
Starting
[5.07340513]
Starting
[5.75979367]
Likelihood fit function called
('Time Difference before -', -182.81380571397827)


/cvmfs/icecube.opensciencegrid.org/py2-v3.1.1/RHEL_7_x86_64/lib/python2.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in log
/cvmfs/icecube.opensciencegrid.org/py2-v3.1.1/RHEL_7_x86_64/lib/python2.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in log


('Time Difference before -', -164.826869106105)
Starting
[5.30288155]
Starting
[7.21803402]
Starting
[7.45437772]
Starting
[8.24056047]
Starting
[7.16548026]
Starting
[8.05662711]


In [ ]:
from icecube import icetray, dataio, dataclasses, simclasses, clsim
from icecube.icetray import I3Units, OMKey, I3Frame
from icecube.dataclasses import ModuleKey
from os.path import expandvars
import numpy as np
#import matplotlib.pylab as plt
#import argparse
from scipy import stats
from scipy.optimize import minimize
from scipy.stats.distributions import chi2
import scipy
from likelihoodRatioAnalysis import likelihoodfit

#file = dataio.I3File(str(infile))
#parser = argparse.ArgumentParser(description = "Likelihood analysis")
#parser.add_argument('-cmin', '--min_charge', dest = 'cmin', help= 'Minimum charge of the events')
#parser.add_argument('-cmax', '--max_charge', dest = 'cmax', help= 'Maximum charge of the events')
#args = parser.parse_args()

gcd_file = dataio.I3File('/home/users/akatil/P-ONE/GCD_files/PONE_Phase1.i3.gz')
cframe = gcd_file.pop_frame()
geometry = cframe["I3Geometry"]
omgeo = geometry.omgeo
print('loaded geometry')

tauLLR = ([])
eLLR = ([])

tauLLR2 = ([])
eLLR2 = ([])

tauTimeDiff = ([])
eTimeDiff = ([])

tauTimeDiff2 = ([])
eTimeDiff2 = ([])

rejected = ([])
for i in range(0, 2000):
    print(i)
    file = dataio.I3File('/data/p-one/akatil/step_4_medium_water/NuTau_NuE_20Events/step_4_'+str(i)+'_medium_water_custom_mDOM_noise.i3.gz')

    for frame in file:
        print('Starting')
        numHitsinDOM = ([])
        numHits = ([])
        mctree = frame["I3MCTree"]
        primary = mctree.primaries
        lepton = dataclasses.I3MCTree.first_child(mctree, primary[0].id)

        mcpeMap = frame['MCPESeriesMap']
        noiseMap = frame['NoiseSeriesMap']

        #looping through doms that have physics hits
        for omkey in mcpeMap.keys():
            oKey = omgeo.get(omkey)

            '''
            Obtaining the timeList
            '''
            noise_mcpeList = noiseMap[omkey]
            noise_timeList = np.array([mcpe.time for mcpe in noise_mcpeList])
            mcpeList = mcpeMap[omkey]
            timeList = np.array([mcpe.time for mcpe in mcpeList])
            tot_timeList = np.append(timeList, noise_timeList)
            numHitsinDOM = np.append(tot_timeList, noise_timeList)

            #print(numHitsinDOM)

        numHits = np.append(numHits, sum(numHitsinDOM))
        log_numHits = np.log10(numHits[numHits > 0])
        print(log_numHits)
        if log_numHits >= float(9.5) and log_numHits < float(10):
            val, timeDiff, val2, timeDiff2, string = likelihoodfit(frame, omgeo)
            if string == 'tau':
                tauLLR = np.append(tauLLR, val)
                tauLLR2 = np.append(tauLLR2, val2)
                tauTimeDiff = np.append(tauTimeDiff, timeDiff)
                tauTimeDiff2 = np.append(tauTimeDiff2, timeDiff2)
            if string == 'e':
                eLLR = np.append(eLLR, val)
                eLLR2 = np.append(eLLR2, val2)
                eTimeDiff = np.append(eTimeDiff, timeDiff)
                eTimeDiff2 = np.append(eTimeDiff2, timeDiff2)
            if string == 'rejected events':
                rejected = np.append(rejected, val)

loaded geometry
0
Starting
[5.33727169]
Starting
[5.5625788]
Starting
[4.88557688]
Starting
[4.9967984]
Starting
[8.66875003]
Starting
[5.68705231]
Starting
[5.40682639]
Starting
[8.79263322]
Starting
[5.14676398]
Starting
[8.85459341]
Starting
[5.45707489]
Starting
[5.5450921]
Starting
[8.30604514]
Starting
[4.88026072]
Starting
[5.26333005]
Starting
[5.26288539]
1
Starting
[5.02375786]
Starting
[8.32451256]
Starting
[6.39512534]
Starting
[7.92016177]
Starting
[4.17715353]
Starting
[5.20154682]
Starting
[5.05308756]
Starting
[5.02010919]
Starting
[5.49466588]
Starting
[4.67057161]
Starting
[8.1514889]
Starting
[7.95092167]
Starting
[7.5522464]
Starting
[6.74650305]
Starting
[8.46184613]
Starting
[4.37621352]
Starting
[8.13258792]
2
Starting
[5.90510605]
Starting
[9.04072742]
Starting
[6.12718781]
Starting
[5.88048809]
Starting
[8.61699692]
Starting
[6.97733689]
Starting
[8.82979012]
Starting
[5.62241357]
Starting
[5.89563644]
Starting
[9.09217312]
Starting
[8.51159385]
Starting
[5.816

[6.36803036]
Starting
[8.17087304]
Starting
[6.3805751]
Starting
[5.59494833]
Starting
[8.7654159]
Starting
[5.99263413]
Starting
[5.76860753]
Starting
[6.74702257]
Starting
[5.74647259]
Starting
[5.5091758]
Starting
[6.36940987]
Starting
[8.06631432]
Starting
[8.89244737]
Starting
[5.70593779]
25
Starting
[6.49911206]
Starting
[5.54500267]
Starting
[5.9563293]
Starting
[6.03171776]
Starting
[9.17299338]
Starting
[6.45412887]
Starting
[5.40040161]
Starting
[5.65199709]
Starting
[7.72964589]
Starting
[5.58137196]
Starting
[5.59736614]
Starting
[5.31558231]
Starting
[8.59936848]
Starting
[5.48055917]
Starting
[5.30467068]
Starting
[6.1309946]
26
Starting
[5.23331312]
Starting
[8.70642429]
Starting
[7.70108203]
Starting
[4.86077407]
Starting
[6.44114697]
Starting
[4.38223907]
Starting
[8.59617527]
Starting
[5.64551599]
Starting
[8.00148233]
Starting
[7.13421472]
Starting
[8.7141647]
Starting
[6.21268309]
Starting
[5.28387519]
27
Starting
[7.86158719]
Starting
[5.51747556]
Starting
[7.9513

Starting
[8.49553953]
Starting
[5.894879]
Starting
[5.11555255]
Starting
[8.61150233]
Starting
[5.7313898]
Starting
[5.50174773]
Starting
[5.33678027]
Starting
[8.54744948]
Starting
[6.50218142]
49
Starting
[7.7104172]
Starting
[7.47770675]
Starting
[7.44238089]
Starting
[8.47287417]
Starting
[5.97365508]
Starting
[5.4258986]
Starting
[4.05090421]
Starting
[4.22966421]
Starting
[7.45917932]
Starting
[7.4117909]
Starting
[4.32884049]
Starting
[5.88923039]
Starting
[7.57991887]
Starting
[5.03665283]
Starting
[7.93169555]
50
Starting
[8.9196947]
Starting
[5.63579229]
Starting
[5.50926106]
Starting
[8.28458242]
Starting
[5.61613648]
Starting
[8.4951025]
Starting
[8.66952657]
Starting
[7.34598337]
Starting
[8.5154963]
Starting
[6.38994103]
Starting
[5.72083085]
Starting
[5.49271735]
Starting
[9.22639605]
Starting
[8.796361]
51
Starting
[5.6823637]
Starting
[8.58114539]
Starting
[8.28815627]
Starting
[4.87893694]
Starting
[5.18998066]
Starting
[8.25171627]
Starting
[7.89430795]
Starting
[5.6

[8.44266919]
Starting
[5.35413027]
Starting
[5.96067615]
Starting
[5.0627952]
Starting
[8.20005771]
Starting
[6.23761588]
Starting
[5.21644615]
Starting
[8.75947701]
Starting
[8.49544968]
Starting
[5.91594115]
Starting
[5.2160341]
Starting
[5.85734511]
Starting
[6.04632373]
Starting
[8.59008669]
73
Starting
[7.65098287]
Starting
[5.11482403]
Starting
[8.00541647]
Starting
[8.19830801]
Starting
[5.14510529]
Starting
[8.25915811]
Starting
[5.43647942]
Starting
[5.0825764]
Starting
[8.5282967]
Starting
[7.60441512]
Starting
[5.58899264]
Starting
[4.87304749]
Starting
[8.4650205]
Starting
[4.78120694]
Starting
[8.32625307]
Starting
[8.09075239]
Starting
[7.6251231]
Starting
[8.08635095]
74
75
Starting
[7.84506266]
Starting
[5.56546197]
Starting
[5.80406061]
Starting
[4.75253845]
Starting
[4.73432566]
Starting
[7.15065464]
Starting
[4.23878551]
Starting
[5.15996951]
Starting
[5.09943176]
Starting
[5.50850291]
Starting
[5.66005998]
Starting
[7.78532708]
Starting
[8.20492043]
Starting
[5.0207

/cvmfs/icecube.opensciencegrid.org/py2-v3.1.1/RHEL_7_x86_64/lib/python2.7/site-packages/scipy/optimize/_minimize.py:394: RuntimeWarning: Method Powell cannot handle constraints nor bounds.
  RuntimeWarning)


...

[8.19949193]
Starting
[5.65466441]
Starting
[7.09186075]
Starting
[6.94964335]
459
Starting
[8.46820463]
Starting
[4.97662185]
Starting
[6.90311245]
Starting
[7.47543004]
Starting
[5.46606729]
Starting
[7.91507589]
Starting
[5.278755]
Starting
[9.11320517]
Starting
[6.49518734]
Starting
[8.95320507]
Starting
[5.06033651]
Starting
[6.88255599]
Starting
[5.43318388]
Starting
[7.88000102]
Starting
[6.85451711]
Starting
[5.67476727]
Starting
[8.26926586]
460
Starting
[5.3763902]
Starting
[8.90497381]
Starting
[8.05052136]
Starting
[8.59016862]
Starting
[8.22900717]
Starting
[8.42040483]
Starting
[5.46688946]
Starting
[8.82011611]
Starting
[5.45268816]
Starting
[8.78732301]
Starting
[8.38929122]
Starting
[8.354213]
Starting
[7.86699495]
Starting
[8.74398056]
461
Starting
[6.13461704]
Starting
[8.68336357]
Starting
[5.62656115]
Starting
[8.68746591]
Starting
[5.70377376]
Starting
[5.76978249]
Starting
[6.06634997]
Starting
[8.93429015]
Starting
[6.05285091]
Starting
[8.9995757]
Starting
[9.0

[8.88748016]
Starting
[8.60249595]
Starting
[8.75356542]
Starting
[9.25020925]
Starting
[8.0041231]
Starting
[6.21460453]
Starting
[8.69778142]
Starting
[5.89899028]
Starting
[5.55696367]
Starting
[8.56355554]
Starting
[7.83067543]
Starting
[7.79958626]
Starting
[5.58767549]
Starting
[8.56262394]
Starting
[5.97053729]
Starting
[5.9926396]
Starting
[8.96795631]
Starting
[5.26620267]
483
Starting
[7.72784336]
Starting
[8.56495352]
Starting
[7.73883554]
Starting
[8.22989456]
Starting
[4.57597179]
Starting
[8.7140749]
Starting
[5.26776262]
Starting
[6.88962742]
Starting
[4.96351491]
Starting
[7.67766999]
Starting
[7.39887508]
Starting
[5.02725266]
Starting
[5.33532928]
Starting
[7.70081172]
Starting
[7.07189798]
Starting
[4.82749315]
Starting
[6.89299]
484
Starting
[4.76849153]
Starting
[7.32206749]
Starting
[5.42075298]
Starting
[4.94737368]
Starting
[7.40080053]
Starting
[7.43074717]
Starting
[8.02309202]
Starting
[7.77533179]
Starting
[4.82481958]
Starting
[8.20431383]
Starting
[4.85351

[5.51568624]
Starting
[4.78972252]
Starting
[8.29864449]
Starting
[5.38675758]
Starting
[5.04247516]
Starting
[8.12143039]
Starting
[7.95031677]
Starting
[8.43660698]
Starting
[6.62258929]
Starting
[8.0148139]
506
Starting
[8.68622153]
Starting
[5.66953756]
Starting
[8.52126689]
Starting
[8.51335455]
Starting
[7.85943485]
Starting
[8.17174642]
Starting
[8.38929347]
Starting
[5.97761873]
Starting
[8.79698481]
Starting
[4.92887862]
Starting
[7.29417975]
Starting
[5.41291894]
Starting
[5.58304357]
Starting
[5.75425713]
Starting
[9.0483919]
507
Starting
[8.53874425]
Starting
[5.25005296]
Starting
[8.07261361]
Starting
[5.49774181]
Starting
[4.20287602]
Starting
[8.46637492]
Starting
[5.33470842]
Starting
[4.78653487]
Starting
[8.0738672]
Starting
[4.54328801]
Starting
[9.2471183]
Starting
[7.38400872]
Starting
[4.24487901]
Starting
[8.08464156]
Starting
[4.10695742]
508
Starting
[8.63794034]
Starting
[8.78538729]
Starting
[7.35189098]
Starting
[5.26350204]
Starting
[5.81019375]
Starting
[5

[5.22324308]
Starting
[5.47583353]
Starting
[6.0013341]
Starting
[8.94687993]
Starting
[4.69107079]
Starting
[5.49684876]
Starting
[8.65857883]
Starting
[8.35493315]
Starting
[8.39443886]
Starting
[8.23731597]
Starting
[5.67400036]
Starting
[5.19960838]
Starting
[8.01480366]
532
Starting
[5.79323809]
Starting
[5.1692802]
Starting
[4.12417881]
Starting
[5.37750319]
Starting
[8.27543425]
Starting
[8.80081849]
Starting
[8.59217618]
Starting
[5.31932464]
Starting
[8.82184548]
Starting
[4.5337099]
Starting
[8.49775454]
Starting
[5.41470401]
Starting
[8.51112762]
Starting
[5.52102898]
Starting
[5.32883179]
Starting
[7.40114995]
Starting
[5.120224]
533
Starting
[4.12334976]
Starting
[5.16495677]
Starting
[4.56399116]
Starting
[7.88370398]
Starting
[4.43579342]
Starting
[7.48115353]
Starting
[8.15245369]
Starting
[7.78580183]
Starting
[6.99464321]
Starting
[7.93382744]
Starting
[4.77326357]
Starting
[5.68583152]
534
Starting
[9.35423225]
Starting
[5.58561933]
Starting
[7.25848814]
Starting
[9.

[5.07701699]
Starting
[9.13371996]
Starting
[5.90437655]
Starting
[5.56633764]
Starting
[5.71301667]
Starting
[6.17872205]
Starting
[8.83034156]
Starting
[8.55114784]
Starting
[5.8074932]
Starting
[5.70066945]
Starting
[5.7375322]
Starting
[5.39889623]
Starting
[5.13935924]
Starting
[7.81605338]
Starting
[5.21375041]
Starting
[5.79038289]
556
Starting
[5.81435689]
Starting
[5.73926507]
Starting
[7.91972484]
Starting
[7.94228193]
Starting
[5.50340199]
Starting
[9.33423635]
Starting
[8.70209601]
Starting
[6.44436067]
Starting
[9.00327274]
Starting
[5.83080929]
Starting
[5.50387736]
Starting
[5.46348672]
Starting
[8.52789778]
Starting
[7.62100307]
Starting
[6.45295367]
Starting
[5.84829753]
Starting
[5.50780711]
Starting
[5.48353939]
557
Starting
[5.9064784]
Starting
[5.35937711]
Starting
[7.97760785]
Starting
[5.27915863]
Starting
[8.84897916]
Starting
[8.88455665]
Starting
[5.67001621]
Starting
[8.63351599]
Starting
[6.11143662]
Starting
[9.00658336]
Starting
[6.06530118]
Starting
[8.88